In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [5]:
fake = pd.read_csv("Fake.csv")
true = pd.read_csv("True.csv")

In [6]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [7]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [8]:
fake['class']=0
true['class']=1

In [9]:
data = pd.concat([fake,true], axis = 0)

In [10]:
data.sample(10)

,title,text,subject,date,class
16342,Cambodian PM says sanctions wouldn't hurt him,PHNOM PENH (Reuters) - Cambodian Prime Ministe...,worldnews,"October 27, 2017",1
16977,Italy's Renzi pledges to hike budget deficit i...,ROME (Reuters) - Former Prime Minister Matteo ...,worldnews,"October 19, 2017",1
2679,Trump to Republican senators: Don't leave town...,WASHINGTON (Reuters) - U.S. President Donald T...,politicsNews,"July 19, 2017",1
2961,"Collins says Senate healthcare bill needs ""maj...",WASHINGTON (Reuters) - Republican Senator Susa...,politicsNews,"June 29, 2017",1
18781,GET OFF OUR CAMPUS! How Universities Plan To “...,United States universities are obsessed with h...,left-news,"Apr 21, 2017",0
7928,Proposed U.S. anti-inversion rules enter final...,"WASHINGTON (Reuters) - U.S. regulations, propo...",politicsNews,"October 4, 2016",1
15384,ZIMBABWE’S CORRUPT GOVERNMENT SEEKS EXTRADITIO...,So much outrage by so many corrupt people it s...,politics,"Jul 31, 2015",0
19147,#BoycottGrammys… #TurnOffTheGrammys…Grammys Pr...,In a fraught moment of political protest acros...,left-news,"Feb 12, 2017",0
17940,Zimbabwe's Mugabe creates cyber ministry in ca...,HARARE (Reuters) - Zimbabwe President Robert M...,worldnews,"October 9, 2017",1
10453,Virginia becomes first U.S. state to regulate ...,(Reuters) - Virginia Governor Terry McAuliffe ...,politicsNews,"March 8, 2016",1


In [11]:
data = data.drop(["title", "subject", "date"], axis=1)

In [12]:
data.reset_index(inplace=True)

In [13]:
data.drop(["index"], axis=1, inplace=True)

In [14]:
data.sample(5)

,text,class
38618,UNITED NATIONS (Reuters) - U.N. Secretary-Gene...,1
20180,The domestic terror group is at it again Watch...,0
836,Just imagine the deafening outrage from conser...,0
39976,STRASBOURG/BRUSSELS (Reuters) - EU lawmakers a...,1
30486,WASHINGTON (Reuters) - President-elect Donald ...,1


In [15]:
def clean_text(text):
    text = text.lower()
    text = re.sub("\[.*?\]", "", text)
    text = re.sub("\\W", " ", text)
    text = re.sub("https?://\S+|www\.\S+", "", text)
    text = re.sub("<.*?>+", "", text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub("\n", "", text)
    text = re.sub("\w*\d\w*", "", text)
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Tanjina Hasan\AppData\Local\Temp\ipykernel_6380\3052142133.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub("\[.*?\]", "", text)
C:\Users\Tanjina Hasan\AppData\Local\Temp\ipykernel_6380\3052142133.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub("https?://\S+|www\.\S+", "", text)
C:\Users\Tanjina Hasan\AppData\Local\Temp\ipykernel_6380\3052142133.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub("\w*\d\w*", "", text)


In [16]:
data['text'] = data['text'].apply(clean_text)

In [17]:
x = data['text']
y = data['class']

xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [18]:
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

In [19]:
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,100
,multi_class,'deprecated'


In [20]:
prediction = lr.predict(xv_test)
lr.score(xv_test, ytest)

0.9858351893095768

In [21]:
print(classification_report(ytest,prediction))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [22]:
vectorizer = joblib.load("vectorizer.jb")
model = joblib.load("lr_model.jb")